# PyJive workshop: buckling
This workshop is about analysis of buckling problems with `pyjive`. Two different modules are applied, the `LinBuckModule` for linear buckling analysis and the `NonlinModule` for incrementally-iterative geometrically nonlinear analysis. The main model that is used is the `FrameModel`. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import os
import sys

pyjivepath = '../../../pyjive/'
sys.path.append(pyjivepath)

if not os.path.isfile(pyjivepath + 'utils/proputils.py'):
    print('\n\n**pyjive cannot be found, adapt "pyjivepath" above or move notebook to appropriate folder**\n\n')
    raise Exception('pyjive not found')

from utils import proputils as pu
import main
from names import GlobNames as gn

%matplotlib widget

In [2]:
# download additional files (if necessary)

import contextlib
from urllib.request import urlretrieve

def findfile(fname):
    url = "https://gitlab.tudelft.nl/cm/public/drive/-/raw/main/buckling/" + fname + "?inline=false"
    if not os.path.isfile(fname):
        print(f"Downloading {fname}...")
        urlretrieve(url, fname)

## 1. Euler Beam
As a first example, Euler buckling is considered. 

<center><img src="https://raw.githubusercontent.com/fmeer/public-files/main/ciem5110/eulerBeam.png" alt="Euler beam" width="400"/></center>



In [ ]:
findfile("beam.geom")
findfile("eulerLB.pro")
findfile("eulerLoadControl.pro")
findfile("eulerDispControl.pro")

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 1a: Perform linear buckling analysis </b>   
    
1. Which action is used for the matrix assembly? 
2. Print the first 5 buckling values (they are stored in a sorted array in `globdat` with the key `gn.LBFACTORS`)
3. Compare the obtained values to the analytical solution $F=n^2\pi^2EI/L^2$. What can be done to bring the results closer to the theoretical ones? 
   
</p>
</div>

## 2. Frame buckling

## 3. Imperfection sensitivity

In [ ]:
props = pu.parse_file('plasticity.pro')

globdat = main.jive(props)

The material model has a history variable $\kappa$, which is a measure for accumulated plastic strain:

    $$\kappa(t)=\displaystyle\int_0^t\sqrt{\frac{2}{3}\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t):\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t)}\,dt$$

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 1b: Postprocessing </b>   
    
Analyze the results with the tools included in the cell below.
    
1. It is possible to show the evolution of displacements with the `ViewModule`. This can module can be constructed in the notebook. How can you see from the evolution of the displacements that the material behavior is nonlinear?
2. From `J2Material`, we store in `globdat` the maximum value of the history variable $\kappa$ throughout the whole domain $\Omega$ after every time step. We can then plot this to see how $\kappa$ is evolving in time. Do the results make sense?
   
</p>
</div>

In [ ]:
# visualize displacements
view = globdat[gn.MODULEFACTORY].get_module('View','view')

props['view'] = {}
props['view']['plot'] = 'solution[dx]'
props['view']['deform'] = 1
props['view']['ncolors'] = 100
props['view']['interactive'] = 'True'
props['view']['colorMap'] = 'plasma_r'

view.init(props, globdat)
status = view.shutdown(globdat)

# plot history variable as function of time
plt.figure()
plt.xlabel('Time step')
plt.ylabel('$\max_\Omega(\kappa)$')
plt.plot(range(len(globdat['maxkappa'])),globdat['maxkappa'])